# Imports

In [1]:
from postal.parser import parse_address
import json
from duckdb.sqltypes import VARCHAR
import pandas as pd
import re
from unidecode import unidecode
import sys
import os
sys.path.append(os.path.abspath(".."))
sys.path.append(os.path.abspath("../merge_tables"))
from merge_tables.db.connection import connect_to_postgres_via_duckdb
from merge_tables.db.tables import create_clean_account_name_macro

# duck init

In [3]:
duck = connect_to_postgres_via_duckdb()
create_clean_account_name_macro(duck)

✓ Successfully connected DuckDB to PostgreSQL database 'medisoft'
✓ Created clean_account_name macro


In [4]:
def udf_parse_address(address: str) -> str:
    parsed = parse_address(address)
    components = {component: value for value, component in parsed}
    return json.dumps(components)

In [5]:
def udf_clean_german_road(text):
    """Clean German road names for matching."""
    if pd.isna(text) or text == '':
        return None
    text = text.lower().strip()
    text = text.replace('ß', 'ss')
    text = re.sub(r'(str\.|str$|str\s|straße|strasse)', '', text)
    text = unidecode(text)
    text = re.sub(r'[^a-z0-9]', '', text)
    return text


In [6]:
duck.create_function("udf_parse_address", udf_parse_address, [VARCHAR], VARCHAR)
duck.create_function("udf_clean_german_road", udf_clean_german_road, [VARCHAR], VARCHAR)

# name matching

## easybill rows

In [8]:
duck.sql("""
         select distinct on("Kontakt: Kundennummer") * from read_csv('/Users/adrienblanquer/Downloads/easybill_contacts_inv_rec_activ.csv') 
         """)

┌─────────────────────┬───────────────────────┬────────────────────────────┬────────────────────┬─────────────────┬─────────────────────────────────┬────────────────┬──────────────────┬───────────────┬───────────────────────┬────────────────────────────────────────────────────────────────────────┬─────────────────────────────────┬───────────────────────┬───────────────────┬─────────────────────┬───────────────┬──────────────────────┬───────────────────┬────────────────────────────────┬───────────────────────┬──────────────────────────────┬────────────────────────┬───────────────────────────────┬─────────────────────┬────────────────────┬──────────────┬───────────────────────┬──────────────────────────────────────────┬──────────────────────────┬───────────────────┬─────────────────┬─────────────────┬─────────────────────┬──────────────────────────────┬──────────────────────────┬─────────────────────────┬───────────────────────┬───────────────────┬──────────────────────┬─────────────────

## eb / medisoft name matched

In [23]:
duck.sql(
    """
    with clean_medisoft as (
        select 
            *,
            clean_account_name(coalesce(name, kuerzel)) as clean_name
        from pg.medisoft.table_firmenstruktur
    ), clean_easybill as (
        select 
            *,
            clean_account_name(trim(unnest(split("Kontakt: Firma", '/')))) as clean_name
        from read_csv('/Users/adrienblanquer/Downloads/easybill_contacts_inv_rec_activ.csv')

    )
    select 
        m.rec_id as medisoft_id,
        easybill."Kontakt: Kundennummer" as easybill_customer_number,
        easybill."Kontakt: Firma" as easybill_name,
        easybill.clean_name as clean_easybill_name,
        coalesce(kuerzel, name) as medisoft_name,
        m.clean_name as clean_medisoft_name,
        jaro_winkler_similarity(clean_account_name(easybill."Kontakt: Firma"), m.clean_name) as sim

    from clean_easybill as easybill
    join clean_medisoft as m
        on jaro_winkler_similarity(clean_account_name(easybill."Kontakt: Firma"), m.clean_name) > 0.95
        or (easybill.clean_name in m.clean_name or m.clean_name in easybill.clean_name and jaro_winkler_similarity(clean_account_name(easybill."Kontakt: Firma"), m.clean_name) > 0.6)
    QUALIFY row_number() OVER (PARTITION BY easybill."Kontakt: Kundennummer" ORDER BY sim DESC) = 1
    """
).to_csv('output/easybill_medisoft_name_matched.csv')

# address and name matching

## eb/medisoft matched rows

In [47]:
duck.sql(
    """
    with raw_medisoft_add as (
        select
            rec_id,
            coalesce(name, kuerzel) as name,
            concat_ws(' ', strasse, plz, 'Deutschland') as street_address
        from pg.medisoft.table_firmenstruktur
    ), medisoft_all_raw_parsed_add as (
        select
            *,
            udf_parse_address(street_address)::json as parsed_address
        from raw_medisoft_add
    ), medisoft_all_parsed_add as (
        select
            *,
            parsed_address->'house_number' as house_number,
            parsed_address->'road' as road,
            udf_clean_german_road(parsed_address->'road') as clean_road,
            parsed_address->'city' as city,
            parsed_address->'postcode' as postcode
        from medisoft_all_raw_parsed_add
    ), easybill_raw_add as (
        select distinct on("Kontakt: Kundennummer")
            "Kontakt: Kundennummer" as eb_id,
            "Kontakt: Firma" as eb_name,
            "Kontakt: Straße/Hausnummer" as eb_nb_street,
            "Kontakt: Postleitzahl" as eb_postcode,
            *
        from read_csv('/Users/adrienblanquer/Downloads/easybill_contacts_inv_rec_activ.csv')
    ), easybill_all_parsed_add as (
        select
            eb_id,
            eb_name,
            parsed_address->'house_number' as house_number,
            parsed_address->'road' as road,
            udf_clean_german_road(parsed_address->'road') as clean_road,
            parsed_address->'postcode' as postcode,
            parsed_address->'country' as country,
        from (select *, udf_parse_address(eb_nb_street || ' ' || eb_postcode || ', Deutschland')::json as parsed_address from easybill_raw_add) as e
    )
    select *,
        jaro_winkler_similarity(easybill_all_parsed_add.clean_road, medisoft_all_parsed_add.clean_road) as name_sim
    from easybill_all_parsed_add
    join medisoft_all_parsed_add
        on (jaro_winkler_similarity(easybill_all_parsed_add.clean_road, medisoft_all_parsed_add.clean_road) > 0.95
        and easybill_all_parsed_add.postcode = medisoft_all_parsed_add.postcode
        and jaro_winkler_similarity(easybill_all_parsed_add.house_number, medisoft_all_parsed_add.house_number) > 0.95)       
    --where Id = '386758000035779091'
    --where Id IS NULL
    qualify row_number() over (partition by eb_id order by name_sim desc) = 1
    order by eb_id

    """
)

┌───────────┬───────────────────────────────────────────────────────────────────────┬──────────────┬─────────────────────────┬───────────────────┬──────────┬───────────────┬──────────────────────────────────────┬──────────────────────────────────────────────────────────────────────────────────────┬─────────────────────────────────────────────────────────────────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬──────────────┬─────────────────────────┬──────────────────┬──────┬──────────┬────────────────────┐
│   eb_id   │                                eb_name                                │ house_number │          road           │    clean_road     │ postcode │    country    │                rec_id                │                                         name                                         │                             street_address                              │     

## merge from the zoho x medisoft

In [53]:
duck.sql(
    """
    select count(*) from read_csv('output/easybill_medisoft_name_matched.csv')
    join pg.medisoft.table_firms_zoho as mz
        on medisoft_id = mz.rec_id
    union all
    select count(*) from read_csv('output/easybill_medisoft_name_matched.csv')
    """
)

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│          376 │
│          376 │
└──────────────┘

All matches from easybill x medisoft are already in the medisoft x zoho table

**So no need to use this matching**

In [60]:
duck.sql(
    """
    select count(distinct mf.rec_id) 
    from read_csv('output/easybill_zoho_final_matches.csv') as ebz
    join pg.medisoft.table_firms_zoho as mz
        on ebz.zoho_id = mz.id_zoho
    join pg.medisoft.table_firmenstruktur as mf
        on mz.rec_id = mf.rec_id
    """
)

┌───────────────────────────┐
│ count(DISTINCT mf.rec_id) │
│           int64           │
├───────────────────────────┤
│                       398 │
└───────────────────────────┘

## not matched

In [32]:
duck.sql(
    """
    with clean_medisoft as (
        select
            *,
            clean_account_name(trim(coalesce(name, kuerzel))) as clean_name
        from pg.medisoft.table_firmenstruktur
    ), clean_easybill as (
        select 
            *,
            clean_account_name(trim("Kontakt: Firma")) as clean_name
        from read_csv('/Users/adrienblanquer/Downloads/easybill_contacts_inv_rec_activ.csv')
    )
    select
        m.rec_id as m_id,
        e."Kontakt: Kundennummer" as e_id,
        coalesce(name, kuerzel) as m_name,
        e."Kontakt: Firma",
        jaro_winkler_similarity(e.clean_name, m.clean_name) as sim,
    from clean_easybill e
    join (select * from clean_medisoft where length(clean_name) > 2) m
        on jaro_winkler_similarity(e.clean_name, m.clean_name) > 0.95
        or (e.clean_name in m.clean_name or m.clean_name in e.clean_name and jaro_winkler_similarity(clean_account_name(e."Kontakt: Firma"), m.clean_name) > 0.6)

    QUALIFY row_number() OVER (PARTITION BY e."Kontakt: Kundennummer" ORDER BY sim DESC) = 1
    order by sim 
    """
)

┌───────────────┬───────────┬────────────────────────────────────────────────────────────┬───────────────────────────────────────────────────────────────────────┬────────────────────┐
│     m_id      │   e_id    │                           m_name                           │                            Kontakt: Firma                             │        sim         │
│    varchar    │   int64   │                          varchar                           │                                varchar                                │       double       │
├───────────────┼───────────┼────────────────────────────────────────────────────────────┼───────────────────────────────────────────────────────────────────────┼────────────────────┤
│ 00_9EY00PWL0L │ 130000041 │ Diabetologische Schwerpunktpraxis Dr. Med. Gundolf Bennöhr │ P.U.N.K.T. PR GmbH                                                    │ 0.5695611577964519 │
│ 00_9VC00HAZMB │ 130001583 │ Radiologische Praxis Dr. med. Anne Packebusch     

In [105]:
easybill_zoho_df = duck.sql(
    """
    with clean_zoho as (
        select 
            * replace(
                trim(unnest(split(Account_Name, '/'))) as Account_Name
            ),
            clean_account_name(trim(unnest(split(Account_Name, '/')))) as clean_name
        from pg.zoho.Accounts
    ), clean_easybill as (
        select 
            * replace(
                trim(unnest(split("Kontakt: Firma", '/'))) as "Kontakt: Firma"
            ),
            clean_account_name(trim(unnest(split("Kontakt: Firma", '/')))) as clean_name
        from read_csv('/Users/adrienblanquer/Downloads/easybill_contacts_inv_rec_activ.csv')
    )
    select 
        z.Id as zoho_id,
        easybill."Kontakt: Kundennummer" as easybill_customer_number,
        easybill."Kontakt: Firma" as easybill_name,
        easybill.clean_name as clean_easybill_name,
        z.Account_Name as zoho_name,
        z.clean_name as clean_zoho_name,
        jaro_winkler_similarity(clean_account_name(easybill."Kontakt: Firma"), z.clean_name) as sim

    from clean_easybill as easybill

    join (select * from clean_zoho where Account_Name <> '' and length(Account_Name) > 1 and length(clean_name) > 3 and clean_name <> 'test') z
        on jaro_winkler_similarity(clean_account_name(easybill."Kontakt: Firma"), z.clean_name) > 0.95
        or (easybill.clean_name in z.clean_name or z.clean_name in easybill.clean_name and jaro_winkler_similarity(clean_account_name(easybill."Kontakt: Firma"), z.clean_name) > 0.6)
    QUALIFY row_number() OVER (PARTITION BY easybill."Kontakt: Kundennummer" ORDER BY sim DESC) = 1
    """
).df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [110]:
easybill_zoho_df.sort_values(by='sim', ascending=True).to_csv('output/easybill_zoho_matches.csv')

In [20]:
duck.sql("""select distinct on("Kontakt: Kundennummer") * from read_csv('/Users/adrienblanquer/Downloads/easybill_contacts_inv_rec_activ.csv') where "Kontakt: Kundennummer"  not in (select distinct easybill_customer_number from easybill_zoho_df) 
         """)

CatalogException: Catalog Error: Table with name easybill_zoho_df does not exist!
Did you mean "pg_tables"?

In [95]:
easybill_zoho_df.easybill_customer_number.nunique()

746

In [39]:
easybill_zoho_df.sort_values(by='sim', ascending=True)[3:4]

,Kontakt: Firma,zoho_name,sim
423,W&W Pflegedienstleistung MV GmbH,W&W Pflegedienstleistung MV GmbH Intensivpfleg...,0.906667


In [74]:
duck.sql("select * from pg.zoho.Accounts where Id = '386758000010036250'")

┌────────────────────┬─────────────────────────┬──────────────────────────┬────────────────────┬──────────────────────────────────────────────────────┬─────────┬────────┬────────────────┬─────────┬───────────────┬──────────────┬───────────┬──────────────────────────┬───────────┬────────────────┬──────────┬────────────────────┬────────────────────┬───────────────────────────┬───────────────────────────┬──────────┬───────────────┬───────────────────────────┬────────────────┬─────────────────┬──────────────┬───────────────┬────────────────────────┬────────────────┬──────────────┬───────────────┬─────────────────┬──────────────────┬─────────────┬────────────────────┬────────────────┬──────────────┬────────┬────────────────┬────────────────┬────────────────────────────┬───────────────────┬────────────────────────────┬────────┬──────────────┬────────────────────┬──────────────────────────────┬────────────────────────────┬──────────────────────┬─────────────┬────────────────────┬─────────────

In [76]:
duck.sql(
    """
    select * from (
        select Id, trim(unnest(split(Account_Name, '/'))) as Account_Name from pg.zoho.Accounts  where Id = '386758000010036250' order by Account_Name
    ) where Account_Name <> '' and length(Account_Name) > 1
    """
)

┌────────────────────┬────────────────────────────────────────────────────┐
│         Id         │                    Account_Name                    │
│      varchar       │                      varchar                       │
├────────────────────┼────────────────────────────────────────────────────┤
│ 386758000010036250 │ GFKB Gesellschaft für Kampfmittelbeseitigung mbH M │
└────────────────────┴────────────────────────────────────────────────────┘

In [ ]:
duck.sql(
    """
select jaro_winkler_similarity(clean_account_name(name), clean_account_name('WS Wärmeprozesstechnik GmbH')) from pg.medisoft.table_firmenstruktur where rec_id = '00_8TD00S1THI'
    """
)

┌────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ jaro_winkler_similarity(clean_account_name("name"), clean_account_name('WS Wärmeprozesstechnik GmbH')) │
│                                                 double                                                 │
├────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│                                                                                                    1.0 │
└────────────────────────────────────────────────────────────────────────────────────────────────────────┘

In [ ]:
duck.sql(
    """
select * from pg.medisoft.table_firmenstruktur where plz = '80339'"""
)

┌───────────────┬───────────┬───────────┬─────────────────────────┬─────────┬────────────────┬───────────────┬───────────────┬─────────────────┬───────────────┬───────────────┬───────────────┬────────────────┬──────────────────────────────────────────┬────────────┬─────────────────┬─────────┬─────────┬───────────┬────────────┬─────────────────┬───────────────────┬─────────┬─────────┬─────────┬───────────────┬─────────┬─────────┬─────────┬──────────────────┬─────────┬─────────┬─────────┬──────────┬─────────┬───────────────────┬────────────┬─────────────────────────┬────────────┬─────────────┬─────────────┬────────────────────┐
│    rec_id     │  kuerzel  │   name    │          pfad           │ passiv  │ abrechnung_art │ laptop_update │ laptop_delete │ historiepflicht │ statistik_kz1 │ statistik_kz2 │ statistik_kz3 │ sync_timestamp │                sync_hash                 │ sequenz_nr │ farbe_verwenden │ status  │  ebene  │ has_child │ has_child2 │ inoriskrelevant │      strasse      │